In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("themarketplace")

create_database('blp_dataset')
create_table('products_tmp_2')

In [ ]:
original_images = []

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\product")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for inner_folder in os.listdir(folder):
            inner_path = folder = Path(f"{fo}\product\{inner_folder}")

            for file in os.listdir(inner_path):
                if file != "link-to-vendor-page":
                    page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

                if page_content is not None:
                    ## GET DESCRIPTION
                    description = ""
                    desc_content = page_content.find("div", attrs={"class": "vendor-profile-text"})
                    description = get_content(desc_content)
                    
                ## GET TITLE
                title = ""
                raw_title = page_content.find("strong")
                title = get_content(raw_title)
                
                ## GET SELLER
                seller_name = ""
                raw_seller = page_content.find_all("a", href=lambda href: href and "vendor" in href)
                
                if raw_seller:
                    seller_name = get_content(raw_seller[0])
                
                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_2 (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
                    values = (title, description, "The Marketplace", seller_name, "t")

                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid

                    ## GET IMAGE
                    raw_name = page_content.find("img")
                    name = ""

                    if raw_name is not None:
                        try:
                            name = raw_name["src"]
                        except IndexError:
#                             print(f"Image {product_id} has no src")
                            update_image(product_id, "no_image")
                        
                        if name is not "":
                            name = name.replace("http://themarketplace.i2p/", "")
                            url_image = Path(f"{folder_image}\{name}")

                            try:
                                # copy images to new directory, renaming with product id
                                if url_image.exists():
                                    
                                    print(f"Image {product_id} exists... creating")

                                    dst_dir = Path(f"D:\images\p\{product_id}.jpg")
                                    copyfile(url_image, dst_dir)
                                else:
                                    print(f"Image {product_id} does not exists")
                                    update_image(product_id, "no_image")
                            except:
                                print(f"Image {product_id} has error. Except")
                                print(url_image)
                                update_image(product_id, "no_image")
                        else:
                            print(f"Image {product_id} has no name")
                            update_image(product_id, "no_image")